# Dados do repasses da PNAE

#### Recursos repassados pela PNAE Incial:

`7 Colunas:`

1. Co_recursos_repassados
Código identificador do repasse de recursos financeiros.

2. Ano
Ano em que os recursos financeiros foram repassados.

3. Estado
Unidade federativa (UF).

4. Municipio
Nome da cidade ou município ao qual os recursos foram destinados.

5. Esfera_governo
Nível administrativo que a escola está vinculada.

6. Modalidade_ensino
Categoria de ensino à qual os recursos são destinados, tais como:
- Creche
- Educação de Jovens e Adultos (EJA)
- Ensino Fundamental
- Ensino Médio
- Ensino Médio Integral
- Pré-escola
- Atendimento Educacional Especializado (AEE)

7. Vl_total_escolas
Valor total em reais (R$) destinado às escolas.

In [272]:
# Manipulacao dos dados
import pandas as pd
from unidecode import unidecode
import numpy as np

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("PNAE_repasse.csv")
df.shape



(16966695, 7)

#### Retirar dados desnecessarios:

In [273]:
print(df['Modalidade_ensino'].unique())

# Filtrar para manter apenas ENSINO MÉDIO e ENSINO MÉDIO INTEGRAL
df = df[df['Modalidade_ensino'].isin(['ENSINO MÉDIO', 'ENSINO MÉDIO INTEGRAL'])]

# Apenas do rio Grande do sul:
df = df[df['Estado'] == 'RS'] # separa só os do RS 

# Apenas a partit de 2012 e ate 2016:
df = df[df['Ano'] > 2011].reset_index(drop=True) 
df = df[df['Ano'] < 2016].reset_index(drop=True) 

df.head()

['CRECHE' 'EJA' 'ENSINO FUNDAMENTAL' 'ENSINO MÉDIO'
 'ENSINO MÉDIO INTEGRAL' 'PRÉ-ESCOLA'
 'ATENDIMENTO EDUCACIONAL ESPECIALIZADO (AEE)' 'INDÍGENA'
 'MAIS EDUCAÇÃO (PARC. COMPLEM.) - FUNDAMENTAL' 'QUILOMBOLA'
 'MAIS EDUCAÇÃO (PARC. COMPLEM.) - INDÍGENA'
 'MAIS EDUCAÇÃO (PARC. COMPLEM.) - QUILOMBOLA'
 'MAIS EDUCAÇÃO (PARC. COMPLEM.) - MÉDIO']


,Co_recursos_repassados,Ano,Estado,Municipio,Esfera_governo,Modalidade_ensino,Vl_total_escolas
0,576380,2013,RS,AJURICABA,MUNICIPAL,ENSINO MÉDIO,15282
1,576386,2013,RS,ALECRIM,MUNICIPAL,ENSINO MÉDIO,13680
2,576392,2013,RS,ALEGRETE,MUNICIPAL,ENSINO MÉDIO,26200
3,576430,2013,RS,ANTA GORDA,MUNICIPAL,ENSINO MÉDIO,6540
4,576439,2013,RS,ARAMBARE,MUNICIPAL,ENSINO MÉDIO,10440


### 2. Uniformização dos Dados:
- Padronizar valores Categoricos;
- Padronizar valores numéricos.

In [274]:
# Dados categóricos para limpeza:
colunas_to_clean = [
    'Co_recursos_repassados', 'Estado',
    'Municipio', 'Esfera_governo',
]

for col in colunas_to_clean:
    df[col] = (
        df[col].astype(str)
               .str.upper()
               .str.strip()
               .str.replace('-', '', regex=False)
               .str.replace(' ', '_', regex=False)
               .str.replace('�', '/', regex=False)
               .str.replace(r'\s+', ' ', regex=True)
    )

# Dados Numericos:
colunas_to_clean = [
    'Co_recursos_repassados', 'Ano',
    'Vl_total_escolas'
]

for col in colunas_to_clean:
    df[col] = pd.to_numeric(
        df[col].astype(str)
                .str.strip()
                .str.replace(' ', '', regex=False)
                .str.replace(',', '', regex=False),
        errors='coerce'  # substitui valores inválidos por NaN automaticamente
    )

### 3. Análise Exploratoria dos Dados:
- Indentificar a qualidade dos dados; 
- Identificar dados ausentes;
- Tratamento desses Dados.

In [275]:
#Mostra a % de valores faltantes: round(3.141592653589793, 2)
def Faltantes():
    faltantes = df.isna().sum()
    porcentagem = round(((faltantes / len(df)) * 100), 2)
    resultado = pd.DataFrame({
        'Qtd. Faltantes': faltantes,
        '% Faltantes': porcentagem
    }).sort_values(by='% Faltantes', ascending=False)

    print(resultado)

In [276]:
#Dados duplicado nesse data frame
df_exemplo = df.copy()
df_exemplo = df_exemplo.drop(columns=['Co_recursos_repassados']) 
print(f"Linhas totais: {len(df_exemplo)}")
print(f"Duplicadas: {df_exemplo.duplicated().sum()}")
print(f"quantidade de linhas nao Duplicadas: {len(df_exemplo) - df_exemplo.duplicated().sum()}")

Linhas totais: 22796
Duplicadas: 22240
quantidade de linhas nao Duplicadas: 556


Se tirarmos o Codigo de transicoes teremos `MUITOS` registos duplicados, 
pois, a presença de duplicados indica que o Ano não foi considerado ao unir todos os repasses, 
fazendo com que exista varios repasses do PNAE para as escolas durante um ano com o mesmo valor. 

Foi exatemnte isso que aconteceu, se levarmos em conta o codigo das transicoes,
ele mostra que nao tem **nenhum valor duplicado** nesse df

In [277]:
print(f"Duplicadas: {df['Co_recursos_repassados'].duplicated().sum()}")

Duplicadas: 0


Checar o Numero de valores faltantes 

In [278]:
Faltantes()
df.shape

                        Qtd. Faltantes  % Faltantes
Co_recursos_repassados               0          0.0
Ano                                  0          0.0
Estado                               0          0.0
Municipio                            0          0.0
Esfera_governo                       0          0.0
Modalidade_ensino                    0          0.0
Vl_total_escolas                     0          0.0


(22796, 7)

#### Retirar colunas desnecessarias:

In [279]:
df = df.drop(columns=['Co_recursos_repassados']) 

In [280]:
df.describe()

,Ano,Vl_total_escolas
count,22796.000000,2.279600e+04
mean,2013.419065,1.677446e+05
std,1.103777,1.657271e+06
min,2012.000000,3.600000e+02
25%,2012.000000,4.920000e+03
50%,2013.000000,9.810000e+03
75%,2014.000000,2.539500e+04
max,2015.000000,1.977052e+07


### Qualidade dos dados:

`Pontos positivos:` 
- Mais de 860'000 registros (muitos registros).
- Poucos valores ausentes, apenas 1,5% estao faltando.

`Pontos negativos:` 
- Altíssima dispersão porem com uma grande densidade de dados menores . 

#### Outliers:
Por causa da alta dispercao dos dados, nos nao conseguimos indentificar nenhum dado que posso ser considerado um outlier. 

#### Tratamento de dados faltantes. 
Nenhuma linha tem dados faltantes. 

In [281]:
index = df[df['Vl_total_escolas'].isna()].index
#df.drop(index=index, inplace=True)

Faltantes()

                   Qtd. Faltantes  % Faltantes
Ano                             0          0.0
Estado                          0          0.0
Municipio                       0          0.0
Esfera_governo                  0          0.0
Modalidade_ensino               0          0.0
Vl_total_escolas                0          0.0


#### Para visualizar a variação da média dos valores da PNAE ao longo dos anos:
Transforme essa célula em código Python, e execute para gerar um gráfico com a média anual dos valores recebidos pelas escolas:

{
# Agrupar: média de Vl_total_escolas por ano
media_por_ano = df.groupby('Ano')['Vl_total_escolas'].mean()

# Plotar a média por ano (em barras)
media_por_ano.plot(kind='bar')
plt.title('Média do Valor Total por Ano')
plt.xlabel('Ano')
plt.ylabel('Média do valor total recebido pelas escolas')

# Calcular a média geral (para a linha vermelha)
media_geral = df['Vl_total_escolas'].mean()
plt.axhline(media_geral, color='red', linestyle='--', label=f'Média geral ≈ {int(media_geral)}')

plt.legend()
plt.show()
}

Os valores **médios por ano** são muito próximos entre si:
- A variação de altura das barras é pequena.
- Isso sugere baixa granularidade temporal:

Isso nos mostra que o valor total repasse por escola nao teve grandes mudanças do orçamento durante 3 anos.

#### Agrupar os dados:

In [282]:
# Agrupar por Municipio, Ano, Esfera e somar as quantidades
df['Vl_total_escolas'] = pd.to_numeric(df['Vl_total_escolas'], errors='coerce')

df = df.groupby(['Municipio', 'Ano'])['Vl_total_escolas'].sum().reset_index()

print(df)

      Municipio   Ano  Vl_total_escolas
0     AJURICABA  2012            740460
1     AJURICABA  2013            626562
2     AJURICABA  2014            651900
3       ALECRIM  2012            659280
4       ALECRIM  2013            560880
..          ...   ...               ...
547  VILA_MARIA  2015            312420
548   WESTFALIA  2012            248460
549   WESTFALIA  2013            255840
550   WESTFALIA  2014            248460
551   WESTFALIA  2015            282900

[552 rows x 3 columns]


In [283]:
df.to_csv('df2.csv', index=False)

df.describe()

,Ano,Vl_total_escolas
count,552.000000,5.520000e+02
mean,2013.418478,6.927366e+06
std,1.104628,6.862198e+07
min,2012.000000,1.476000e+04
25%,2012.000000,2.017200e+05
50%,2013.000000,3.997500e+05
75%,2014.000000,1.023360e+06
max,2015.000000,8.152801e+08
